In [97]:
import os
import csv
import re
from collections import defaultdict, Counter
import nltk
from nltk import bigrams, trigrams
import random
import numpy as np

In [98]:
def remove_punctuation(word):
    return re.sub(r'[^\w\s@]', ' ', word)

Lettura del trump twitter archive

In [99]:
file_name = 'tweets.csv'
current_dir = os.getcwd()
file_path = os.path.join(current_dir, 'trump_twitter_archive/', file_name)

with open(file_path, 'r', encoding='utf-8') as train:
   righe = train.readlines()[1:]

Pulizia dei tweet

In [100]:
tweets = []
for riga in righe:
   riga = riga.split(',')
   # elimina link e tag ad altri utenti 
   riga[1] = re.sub(r"http\S+", "", riga[1])
   riga[1] = re.sub(r"&\S+", "", riga[1])   
   #riga[1] = re.sub(r"@\S+", "", riga[1])

   tweets.append(riga[1])
print(tweets)

['LOSER! ', 'Most of the money raised by the RINO losers of the so-called “Lincoln Project” goes into their own pockets. With what I’ve done on Judges Taxes Regulations Healthcare the Military Vets (Choice!)  protecting our great 2A they should love Trump. Problem is I BEAT THEM ALL!', '....because they don’t know how to win and their so-called Lincoln Project is a disgrace to Honest Abe. I don’t know what Kellyanne did to her deranged loser of a husband Moonface but it must have been really bad. John Weaver lost big for Kasich (to me). Crazed Rick Wilson....', '....lost for Evan “McMuffin” McMullin (to me). Steve Schmidt  Reed Galvin lost for John McCain Romney’s campaign manager (?) lost big to “O”  Jennifer Horn got thrown out of the New Hampshire Republican Party. They’re all LOSERS but Abe Lincoln Republican is all smiles!', '....get even for all of their many failures. You see these loser types don’t care about 252 new Federal Judges 2 great Supreme Court Justices a rebuilt milit

Creazione dei brigrammi e trigrammi

In [101]:
# Costruzione delle strutture a bigrammi e trigrammi
model_bigrams = defaultdict(lambda: defaultdict(lambda: 0))
model_trigrams = defaultdict(lambda: defaultdict(lambda: 0))
tot_length = 0

for tweet in tweets:
    tot_length += len(tweet)
    tweet = remove_punctuation(tweet)
    words = tweet.split()
            
    # Bigrammi
    for w1, w2 in bigrams(words):
        model_bigrams[w1.lower()][w2.lower()] += 1
    
    # Trigrammi
    for w1, w2, w3 in trigrams(words):
        model_trigrams[(w1.lower(), w2.lower())][w3.lower()] += 1
medium_length = round(tot_length / len(tweets))     #lunghezza media dei tweet di Trump
print(tot_length)
print(model_bigrams)

39735
defaultdict(<function <lambda> at 0x000001F1FC80D790>, {'most': defaultdict(<function <lambda>.<locals>.<lambda> at 0x000001F1FC736820>, {'of': 1, 'others': 1, 'disloyal': 1, 'important': 1, 'people': 2, 'corrupt': 1, 'slanted': 1, 'thought': 1, 'over': 1, 'envied': 1}), 'of': defaultdict(<function <lambda>.<locals>.<lambda> at 0x000001F1FC625700>, {'the': 30, 'a': 4, 'their': 2, 'them': 1, 'bernie': 1, 'massachusetts': 1, 'his': 4, 'south': 1, 'jeb': 1, 'moderate': 1, 'zero': 1, 'relentless': 1, 'michigan': 1, 'everybody': 1, 'control': 1, 'london': 1, 'vicious': 1, 'your': 1, 'campaign': 1, '13': 1, 'trouble': 2, 'big': 1, 'immigration': 1, 'these': 2, 'all': 1, 'scotland': 1, 'defeat': 1, 'philadelphias': 1, 'politics': 2, 'which': 1, 'success': 2, 'haters': 5, 'goldberg': 1, 'losers': 2, 'fate': 6, 'everything': 1, 'ny': 1, 'july': 1, 'this': 1, 'lives': 1, 's': 1, 'yourself': 1, 'blackdog': 1, 'myself': 1, 'time': 2, 'you': 2, 'celebrity': 1, 'after': 1, 'loser': 1, 'forbes'

Calcolo delle probabilità per i bigrammi e i trigrammi

In [102]:
#calcolo probabilità bigrammi
for w1 in model_bigrams:
  tot = sum(model_bigrams[w1].values())
  for w2 in model_bigrams[w1]:
      model_bigrams[w1][w2] /= tot
print(model_bigrams)

defaultdict(<function <lambda> at 0x000001F1FC80D790>, {'most': defaultdict(<function <lambda>.<locals>.<lambda> at 0x000001F1FC736820>, {'of': 0.09090909090909091, 'others': 0.09090909090909091, 'disloyal': 0.09090909090909091, 'important': 0.09090909090909091, 'people': 0.18181818181818182, 'corrupt': 0.09090909090909091, 'slanted': 0.09090909090909091, 'thought': 0.09090909090909091, 'over': 0.09090909090909091, 'envied': 0.09090909090909091}), 'of': defaultdict(<function <lambda>.<locals>.<lambda> at 0x000001F1FC625700>, {'the': 0.2830188679245283, 'a': 0.03773584905660377, 'their': 0.018867924528301886, 'them': 0.009433962264150943, 'bernie': 0.009433962264150943, 'massachusetts': 0.009433962264150943, 'his': 0.03773584905660377, 'south': 0.009433962264150943, 'jeb': 0.009433962264150943, 'moderate': 0.009433962264150943, 'zero': 0.009433962264150943, 'relentless': 0.009433962264150943, 'michigan': 0.009433962264150943, 'everybody': 0.009433962264150943, 'control': 0.0094339622641

In [103]:
#calcolo probabilità trigrammi
for w1w2 in model_trigrams:
    tot = sum(model_trigrams[w1w2].values())
    for w3 in model_trigrams[w1w2]:
        model_trigrams[w1w2][w3] /= tot
print(model_trigrams)

defaultdict(<function <lambda> at 0x000001F1FBC05D30>, {('most', 'of'): defaultdict(<function <lambda>.<locals>.<lambda> at 0x000001F1FBE8EB80>, {'the': 1.0}), ('of', 'the'): defaultdict(<function <lambda>.<locals>.<lambda> at 0x000001F1FBE8E700>, {'money': 0.03333333333333333, 'so': 0.03333333333333333, 'new': 0.03333333333333333, 'night': 0.03333333333333333, 'dumbest': 0.03333333333333333, 'united': 0.06666666666666667, 'people': 0.03333333333333333, 'president': 0.03333333333333333, 'losers': 0.06666666666666667, 'witch': 0.03333333333333333, 'great': 0.06666666666666667, 'fbi': 0.06666666666666667, 'mainstream': 0.03333333333333333, 'political': 0.03333333333333333, 'conservative': 0.03333333333333333, 'haters': 0.16666666666666666, 'world': 0.06666666666666667, 'repetitive': 0.03333333333333333, 'highest': 0.03333333333333333, 'obama': 0.03333333333333333, 'year': 0.03333333333333333}), ('the', 'money'): defaultdict(<function <lambda>.<locals>.<lambda> at 0x000001F1FBE8E4C0>, {'r

### GENERAZIONE DI TWEET

un calcolo per applicare temperatura alle probabilità

In [104]:
def apply_temperature(probabilities, temperature):
    probabilities = np.array(probabilities)
    probabilities = probabilities ** (1.0 / temperature)
    probabilities = probabilities / np.sum(probabilities)
    return probabilities

Tweet basati sui bigrammi

In [105]:
def bi_tweet_generator(length, temperature):
    first_words = list(model_bigrams.keys())
    current_word = random.choice(first_words)
    bi_tweet = [current_word]
    print('Parola iniziale: ')
    print(current_word)
    
    for i in range(length):
        next_words = list(model_bigrams[current_word].keys())
        if next_words:
            probabilities = list(model_bigrams[current_word].values())
            probabilities = apply_temperature(probabilities, temperature)
            new_word = random.choices(next_words, weights=probabilities, k=1)[0]            
        else:
            break   # Non ci sono parole disponibili, esci dal ciclo
        
        bi_tweet.append(new_word)
        current_word = new_word
    return bi_tweet

Tweet basati su trigrammi

In [119]:
def tri_tweet_generator(length, temperature):
    '''
    first_words = list(model_bigrams.keys())
    current_word = random.choice(first_words)
    tri_tweet = [current_word]
    print('Parola iniziale: ')
    print(current_word)
    current_word = 'fold'
    '''
    #alla prima iterazione creiamo un bigramma tweet di due parole con i bigrammi
    tri_tweet = bi_tweet_generator(1, temperature)
    prev_word = tri_tweet[0]
    current_word = tri_tweet[1]
    '''
    new_word = max(model_bigrams[current_word], key=model_bigrams[current_word].get)
    tri_tweet.append(new_word)
    prev_word = current_word
    current_word = new_word
    '''
    #nelle iterazioni successive si usano i trigrammi
    for i in range(length-1):
        next_words = list(model_bigrams[current_word].keys())                
        if model_trigrams[(prev_word, current_word)]:
            next_words = list(model_trigrams[(prev_word, current_word)].keys())
            if next_words:
                probabilities = list(model_trigrams[(prev_word, current_word)].values())
                probabilities = apply_temperature(probabilities, temperature)
                new_word = random.choices(next_words, weights=probabilities, k=1)[0]
            else:
                break
        else:
            break
        tri_tweet.append(new_word)
        prev_word = current_word
        current_word = new_word
        
    return tri_tweet

In [107]:
#al momento il controllo sulla punteggiatura elimina le @ degli account taggati, quindi ci sono parole incomprensibili(i nomi degli account senza @)
#fare prove modificando lunghezza frase
#salvare anche i segni di punteggiatura come parole a sè stanti
#si possono aggiungere controlli (se parola già usata nella frase, non ripeterla)
#calcolare lunghezza media dei tweet originali

In [108]:
# alcuni parameteri
n_tweets = 15
length = medium_length
temperature = 0.7

In [118]:
for i in range(n_tweets):   
    tweet = bi_tweet_generator(length, temperature)
    print("tweet:\t", tweet)
    

Parola iniziale: 
out
tweet:	 ['out', 'at', 'this', 'really', 'dumb', 'haters', 'and', 'get', 'it', 'or', 'accept', 'the', 'haters', 'losers', 'if', 'i', 'never', 'have', 'no', 'name', 'or', 'insecureit', 's', 'not', 'well', 'be', 'presented', 'to', 'go', 'angelo', 'failed', 'and', 'the', 'new', 'twist', 'of', 'the', 'few', 'remaining', 'never', 'went', 'hostile', 'major', 'loser', 'for', 'the', 'world', 'class', 'of', 'the', 'award', 'over', 'scotland', 'yard', 'must', 'be', 'if', 'the', 'fbi', 'ever', 'recover', 'it', 's', 'gone', 'now', 'dismissed', 'or', 'not', 'paltrow', 'loser', '@cpacnews', 'james', 'comey', 'andrew', 'mccabe', 'peter', 's', 'for', 'the', 'hell', 'were', 'too', 'bad', 'ratings', 'on', 'trump', 'nor', 'ever', 'want', 'to', 'try', 'to', 'your', 'face', 'not', 'losers', 'and', 'loser', 'newspaper', 'loser', 'nothing', 'to', 'more', 'than', 'the', 'night', 'in', 'that', 'is', 'a', 'businessman', 'for', 'the', 'losers', 'they', 'are', 'they', 'are', 'a', 'failed', 'h

In [120]:
for i in range(n_tweets):
    tweet = tri_tweet_generator(length, temperature)
    print("tweet:\t", tweet)

Parola iniziale: 
spread


KeyboardInterrupt: 